## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.


Для того, чтобы ответить на  вопрос, поставленный заказчиком.
Ответим на вопросы:
-	Есть ли зависимость между наличием детей и возвратом кредита в срок?
-	Есть ли зависимость между семейным положением и возвратом кредита в срок?
-	Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
-	Как разные цели кредита влияют на его возврат в срок?



## Изучение общей информации

Познакомимся с данными из файла. Для этого: выведем на экран первые 20 строк из таблицы, 

In [1]:
# Загружаем библиотеку:
import pandas as pd
# Используем метод read():
df = pd.read_csv('')
df.head(5) # получение первых 20 строк таблицы df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Посмотрим общую информацию о файле:



In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Дополнительно применим метод columns(), чтобы проверить названия столбцов:

In [3]:
df.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

Полученная таблица состоит из 21525 строк и 12 столбцов, из которых в 7 хранятся количественные переменные: 'children', 'days_employed', 'dob_years', 'education_id', 'family_status_id' , 'total_income', 'debt'; в 5 - категориальные: 'education', family_status',  'gender', 'income_type', 'purpose'.

В столбцах 'days_employed', 'total_income' есть значение NaN, которое указывает на то, что в данных ячейках никакого значения нет. А это значит, что по какой-то причине данные пропущены и никак не будут учтены в расчетах, что, в свою очередь, приведет к некорректным результатам. На наличие пропусков указывает и количество заполненных строк для указанных столбцов :19351. Их меньше по сравнению с общим числом строк.

Поэтому прежде чем приступать к расчетам нужно: выяснить причины пропусков и по возможности заполнить их.

Названия столбцов изменять не нужно. Все названия столбцов на латинском языке, без пробелов, написаны в одном регистре. Пробел между слов заменен на нижнее подчеркивание. Название отражает смысл, хранимых в них данных.  

Описание данных:
-	children — количество детей в семье
-	days_employed — общий трудовой стаж в днях
-	dob_years — возраст клиента в годах
-	education — уровень образования клиента
-	education_id — идентификатор уровня образования
-	family_status — семейное положение
-	family_status_id — идентификатор семейного положения
-	gender — пол клиента
-	income_type — тип занятости
-	debt — имел ли задолженность по возврату кредитов
-	total_income — ежемесячный доход
-	purpose — цель получения кредита

В столбце 'education' данные записаны в разном регистре.
В столбце 'purpose' хранятся данные, имеющие один смысл,но записаны по - разному.
Такие данные нужно обработать, чтобы в дальнейшем удобно было их применять в расчетах.


### Вывод

Количество строк в таблице указывает на количество выданных клиентам кредитов.
Каждая строка содержит информацию о клиенте: пол, семейное положение, количество детей в семье, уровень образования, возраст, тип занятости. А также указана цель получения кредита и была ли задолженность у клиента по возрату кредита в срок.





# Предобработка данных

Чтобы получить ответ на поставленные задачи для расчетов потребуются столбцы: 'children', 'family_status', 'family_status_id',  'debt','total_income', 'purpose'. Поэтому предобработку данных будем осуществлять в указанных столбцах.

### *Обработка пропусков*

Посчитаем количество пропусков с помощью метода isnull() и sum():

In [4]:
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Вычислим какой процент составляют пропуски в столбце 'total_income' от общего количества строк:

In [5]:
total_lines = len(df) #переменная, в которой хранится общее число выданных кредитов
total_skips = 2174    #число пропусков в столбце "ежемесячный доход"
percent = total_skips / total_lines
print('Процент пропущенных значений составляет: {:.0%}'.format(percent))


Процент пропущенных значений составляет: 10%


Удалять строки с пропусками не будем, т.к. процент строк без значений достаточно высокий.


Разделим всех клиентов банка на группы, в зависимости от их типов занятости. Проверим какой процент пропусков составляет в каждой группе.

In [6]:
# скопируем данные с пропусками таблицы df в другую таблицу nan_table
nan_table = df[df['total_income'].isna()].copy()
nan_table.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


In [7]:
#Вычислим какой процент каждой группы составляет в таблице с пропусками nan_table с помощью метода value_counts():
nan_table['income_type'].value_counts(normalize = True)

сотрудник          0.508280
компаньон          0.233671
пенсионер          0.189972
госслужащий        0.067617
предприниматель    0.000460
Name: income_type, dtype: float64

In [8]:
#Вычислим какой процент каждой группы составляет в таблице без пропусков df с помощью метода value_counts():
df['income_type'].value_counts(normalize = True)

сотрудник          0.516562
компаньон          0.236237
пенсионер          0.179141
госслужащий        0.067782
безработный        0.000093
предприниматель    0.000093
студент            0.000046
в декрете          0.000046
Name: income_type, dtype: float64

Результат: большинство клиентов банка имеют тип занятости 'сотрудник'. И ежемесячный уровень дохода не заполнен так же у этой группы людей(51%). И чтобы их заполнить - вычислим медиану для этой группы.

И таким образом поступим для каждой группы.

Создадим новый столбец, в который вместо пропуска будет подставлено значение медианы, исходя из типа занятости клиента, а остальные значения(без пропуска) останутся без изменений:

In [9]:
# Создаем новый столбец 'new_total_income' и в него сохраняем данные из столбца 'total_income':
df['new_total_income'] = df['total_income']
# Для каждого уникального значения из столбца 'income_type':
# 1) находим значение медианы в каждой группе
# 2) Фильтруем строки по условию: в каждой группе оставляем только строки с пропущенными значениями
# 3) Полученный результат заменяем на значение медианы, соответствующей группе:
for income_type in df['income_type'].unique():
    median_income = df[df['income_type'] == income_type]['total_income'].median()
    df.loc[(df['new_total_income'].isna())&
           (df['income_type'] == income_type), 'new_total_income'] = median_income
# Проверяем результат: выводим столбцы income_type, total_income, new_total_income, 
# где total_income - это строки, содержащие пропуски:
# Выводим последние 5 строк:
df[['income_type', 'total_income', 'new_total_income']].query('total_income.isna()').tail(5)

,income_type,total_income,new_total_income
21489,компаньон,NaN,172357.950966
21495,сотрудник,NaN,142594.396847
21497,компаньон,NaN,172357.950966
21502,сотрудник,NaN,142594.396847
21510,сотрудник,NaN,142594.396847


В результате создан новый столбец 'new_total_income'. И в дальнейшем для расчетов будем обращаться к нему. Пропуски заполнены.

Пропуски есть еще и в столбце 'days_employed' -  общий трудовой стаж в днях. Тип данных столбца - float64. Сразу можем сказать, что количество дней не может быть представлено вещественным числом, а только целым. Если, мы выведем столбец на экран, то увидим - отрицательные числа. Из всего вышесказанного следует: возможно при выгрузке информации из баз данных произошла ошибка. Чтобы прояснить ситуацию нужно обратиться к разработчику или человеку, который формировал таблицы.

Для нас информация в столбце 'days_employed' является второстепенной. Её мы не будем использовать в расчетах, поэтому удалим столбец:

In [10]:
#воспользуемся методом drop():
df.drop(['days_employed'], axis='columns', inplace=True)
df.drop(['total_income'], axis='columns', inplace=True) # Столбец 'total_income' также удалим.
df.info() # проверим изменения

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 11 columns):
children            21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
purpose             21525 non-null object
new_total_income    21525 non-null float64
dtypes: float64(1), int64(5), object(5)
memory usage: 1.8+ MB


### Вывод

Причины возникновения пропусков в таблицах могут быть разными. 

У нас в первом случае возникновение пропуска было неслучайно: клиент не указал свой уровень дохода, во втором - возможной причиной стал сбой в системе.

###  *Замена типа данных*

Тип данных в столбце 'total_income' - ежемесячный доход: float64 - представлен в виде числа с плавающей запятой, среди которых есть числа в научном формате(например, 2.265604e+06). Нам для расчетов такая точность чисел не нужна. Поэтому
для удобства расчетов и наглядности полученных результатов переведем тип данных столбца 'total_income'  в целочисленный (int64):

In [11]:
#т.к. нам нужно преобразовать числовой тип данных в числовой - воспользуемся методом astype():
df['new_total_income'] = df['new_total_income'].astype('int')
df['new_total_income'].dtype # удостоверимся, что тип данных переведен с float64 на int64


dtype('int64')

### Вывод

Все нужные нам замены типов данных выполнены. Можем приступать к поиску дубликатов в таблице.

### *Обработка дубликатов*

Проверим данные в таблице на наличие дубликатов. 

In [12]:
# Применим метод duplicated()в сочетании с методом sum(). Получим общее число дубликатов:

df.duplicated().sum()

54

В таблице 54 полностью повторяющихся строк. Возможно совпадений будет больше, если мы применим метод value_counts() для столбцов содержащих категориальные переменные. И устраним ошибки, которые будут найдены с помощью этого метода. 

Напомним, что 'family_status', 'gender', 'education', 'income_type' - это столбцы, содержащие категориальную переменную:

In [13]:
# Применим метод для столбца 'family_status'
print(df['family_status'].value_counts())
#В результате: В исправлениях данные не нуждаются
print('______________')

# Применим метод для столбца 'gender'
print(df['gender'].value_counts())
#В результате: в столбце где указан пол человека одно уникальное значение 'XNA'. Меньше всего в списке мужчин. 
# Поэтому с помощью метода replace изменим значение 'XNA' на 'M'
df['gender'] = df['gender'].replace('XNA', 'M')
#В результате на одного мужчину больше
print('______________')

# Применим метод для столбца 'education'
print(df['education'].value_counts())
#В результате: Все слова в столбце написаны в разных регистрах. Применим метод .str.lower(), чтобы все символы в строке
#привести к  нижнему регистру
df['education'] = df['education'].str.lower()
#Снова применим метод value_counts(), чтобы проверить изменения:
print('Результат после приведения данных к одному регистру: ')
df['education'].value_counts()
print('______________')
# Применим метод для столбца 'income_type'
print(df['income_type'].value_counts())
#В результате: В исправлениях данные не нуждаются


женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64
______________
F      14236
M       7288
XNA        1
Name: gender, dtype: int64
______________
среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64
Результат после приведения данных к одному регистру: 
______________
сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декр

Готово: нужные преобразования выполнены.

In [14]:
#Проверим как выполненные нами исправления повлияли на количество дубликатов
df.duplicated().sum()

71

Как видим, количество совпадений увеличилось. 

Удалим их из таблицы:

In [15]:
#Применим метод drop_duplicates() вместе с методом Reset_index(), чтобы старые индексы были удалены, 
#а новые индексы были переписаны в порядке возрастания, без пропусков
# и перезапишем результат:
df=df.drop_duplicates().reset_index(drop = True)
df.info() # Проверяем результат

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 11 columns):
children            21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
purpose             21454 non-null object
new_total_income    21454 non-null int64
dtypes: int64(6), object(5)
memory usage: 1.8+ MB


### Вывод



Все дубликаты удалены. Строк в таблице осталось 21454.


Возможная причина возникновения дубликатов в данной таблице: у одного клиента банка есть несколько кредитов.


### *Лемматизация*

Обратим внимание на столбец 'purpose' - цель получения кредита. Он потребуется нам для расчетов, которые помогут ответить на вопрос: как разные цели кредита влияют на его возврат в срок? В нем хранятся значения данных, имеющие один смысл, но написанные по-разному, например: "покупка жилья" и "покупка жилья для семьи". С такими данными будет сложно работать. 

Приведем их к одному виду. Для этого используем алгоритм для поиска слов, записанных в разных формах - лемматизацию. В итоге получим список слов, приведенных к его словарной форме.

In [16]:
# Отсортируем данные в столбце 'purpose' по возрастанию:
df.sort_values('purpose')
from pymystem3 import Mystem # импортируем библиотеку с функцией лемматизации
from collections import Counter# вызовем специальный контейнер, чтобы подсчитать количество уникальных значений
m = Mystem()
text = ' '.join(list(df['purpose'])) 
lemmas = m.lemmatize(text)
print(Counter(lemmas))

#из результата  Counter выберем существительные и создадим словарь:
vocabulary = ['недвижимость', 'автомобиль', 'образование', 'свадьба', 'жилье', 'строительство', 'ремонт']

#добавим к столбцу 'purpose' новый столбец с помощью метода apply(). 
#В итоге получим столбец со словами, приведенных к их словарной форме.
conversion_one = df['conversion_one'] = df['purpose'].apply(m.lemmatize)

#создадим функцию, которая для каждого значения  из словаря 'vocabulary' ищет совпадение со значением
#столбца 'conversion_one' и возвращает значение из словаря
def replacement_words(conversion_one):
    for i in vocabulary:
        if i in conversion_one:
            return i
        
#добавим новый столбец, в котором останутся только слова из словаря 'vocabulary'. В качестве аргумента  метода apply() 
#подставим функцию replacement_words
df['conversion_two'] = df['conversion_one'].apply(replacement_words)
(df[['purpose', 'conversion_one', 'conversion_two']].head(20)) # Проверим результат

Counter({' ': 55023, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'подержать': 853, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'приобретение': 461, 'профильный': 436, 'подержанный': 111, '\n': 1})


,purpose,conversion_one,conversion_two
0,покупка жилья,"[покупка, , жилье, \n]",жилье
1,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль
2,покупка жилья,"[покупка, , жилье, \n]",жилье
3,дополнительное образование,"[дополнительный, , образование, \n]",образование
4,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба
5,покупка жилья,"[покупка, , жилье, \n]",жилье
6,операции с жильем,"[операция, , с, , жилье, \n]",жилье
7,образование,"[образование, \n]",образование
8,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",свадьба
9,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",жилье


### Вывод

Все значения в столбце 'purpose' (цель получения кредита) преобразованы. Теперь мы с легкостью сможем разбить все цели на группы: 'недвижимость', 'автомобиль', 'образование', 'свадьба', 'жилье', 'строительство', 'ремонт' и быстро ответим на вопрос: как разные цели кредита влияют на его возврат в срок? 

### *Категоризация данных*


Рассмотрим столбец с ежемесячными доходами  'total_income': у каждого заемщика свой уровень дохода в месяц. Цифры между собой сильно различаются. У одного 20000 в месяц , у другого - 100000 на семью. Нам нужно ответить на вопрос: есть ли зависимость между уровнем дохода и возвратом кредита в срок? Чтобы легче было ответить на него - нужно разбить цифры на категории - группы, объединяющие данные по какому-то признаку. В случае уровня дохода: можно посмотреть каков средний уровень дохода в России по данным Росстата. Но, опять же, нужно учесть что в различных регионах России свои средний уровень дохода. У нас в таблице данных о том где проживают клиенты нет, поэтому посчитаем квантили для данного столбца: 0.25, 0.50, 0,75. И уже, исходя из полученных цифр, будем делить клиентов на группы: 'Низкий', 'Средний', 'Высокий', 'Очень высокий'
Но перед этим посчитаем какой уровень дохода приходится на одного человека в каждой семье клиента банка.

In [17]:
# Если человек 'женат / замужем' или в 'гражданском браке' прибавим 2 к количеству детей(столбец 'children'); 
# остальным - +1.
# Таким образом узнаем сколько в семье человек
# Затем общий ежемесячный доход разделим на количество человек в семье
# Создадим функцию, которая будет возвращать общее число человек в семье:
def test(row): #передаем всю строчку датафрейма
    family_status_id = row['family_status_id'] #  поле датайфрема, с помощью которого хочется создать новое поле
    children = row['children'] # еще одно поле для датафрейма, с которым будем работать
    if family_status_id == 0 or 1:
        return children + 2
    return children + 1

#С помощью метода apply() добавим новый столбец к таблице:
df['new_field'] = df.apply(test, axis = 1)
#print(df.head(5)) # Проверим результат

# Посчитаем уровень дохода на одного человека в каждой семье. Сохраним результат в новом столбце df['total_income_end']
df['total_income_end'] = df['new_total_income'] / df['new_field']
#print(df.head(5))
print(df['total_income_end'].dtypes) # Изменим тип данных в столбце df['total_income_end'] с с float64 на int64
df['total_income_end'] = df['total_income_end'].astype('int')   
print(df['total_income_end'].head(5)) # Проверим результат

float64
0    84625
1    37360
2    72942
3    53525
4    79308
Name: total_income_end, dtype: int64


Для того чтобы рассчитать квантили для данного столбца 'total_income_end': 0.25, 0.50, 0,75 отсортируем его в порядке возрастания.

In [18]:
pd.options.display.float_format = '{:.2f}'.format  # Задаем формат вывода чисел 
quantile_one = df.total_income_end.quantile([.25]) # С помощью метода quantile() вычислили 0.25 квантиль
quantile_two = df.total_income_end.quantile([.50]) # С помощью метода quantile() вычислили 0.50 квантиль
quantile = df.total_income_end.quantile([.75])     # С помощью метода quantile() вычислили 0.75 квантиль
# Выведем полученный результат на экран:
print(quantile_one) 
print(quantile_two)
print(quantile)

0.25   42929.00
Name: total_income_end, dtype: float64
0.50   61364.00
Name: total_income_end, dtype: float64
0.75   86178.00
Name: total_income_end, dtype: float64


Создадим функцию income_func, результатом которой будет группировка столбца 'total_income_end': доход меньше 42929 - низкий, от 42929 до 61364 - средний, от 61364 до 86178 - высокий, выше 86178 - очень высокий

In [19]:
# Назовем функцию income_func, в аргумент которой будем передавать данные столбца total_income_end:
# Далее задаем нужные нам условия:
def income_func(total_income_end):
    if total_income_end < 42929:
        return 'низкий'
    elif 42929 <= total_income_end < 61364:
        return 'средний'
    elif total_income_end >= 86178:
        return 'очень высокий'
    return 'высокий'
# Проверим как работает функция - подставим в качестве аргумента конкретное число:
a = income_func(10000000)
print(a)


очень высокий


Функция работает корректно.
 
Создадим новый столбец, в котором значения - это группировка клиентов банка, исходя из уровня дохода на одного человека в семье.

In [20]:
# Используем метод apply(): 
#берет значения столбца 'total_income_end' и применяет к ним функцию income_func из своего аргумента
df['income_group'] = df['total_income_end'].apply(income_func)
df[['new_total_income', 'total_income_end', 'income_group']].head(5) # Проверим полученный результат

,new_total_income,total_income_end,income_group
0,253875,84625,высокий
1,112080,37360,низкий
2,145885,72942,высокий
3,267628,53525,средний
4,158616,79308,высокий


In [21]:
# Выведем статистику по группам:
df['income_group'].value_counts()

очень высокий    5570
средний          5365
низкий           5362
высокий          5157
Name: income_group, dtype: int64

В итоге: поделили всех заемщиков банка на 4 категории. Видим, что каждая категория содержит приблизительно одинаковое количество пользователей.



В таблице есть еще один столбец с количественными переменными - 'children' - количество детей, который также нужно поделить на категории.

Разделим всех заемщиков банка на три группы в зависимости от количества детей: бездетные, 1-2 ребенка, многодетные.


In [22]:
# проверим сколько уникальных значений в столбце  'children' :
print(df['children'].value_counts())
# В результате: есть значение '-1'  и '20'.
#'-1' заменим на значение '1', а '20' заменим на '0' с помощью метода Replace():
df['children'] = df['children'].replace(-1, 1)
df['children'] = df['children'].replace(20, 0)
# Проверим замену:
df['children'].value_counts()

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


0    14167
1     4855
2     2052
3      330
4       41
5        9
Name: children, dtype: int64

Сгруппируем всех клиентов банка по наличию детей следующим образом: 0 - 'Бездетные'; 1-2 - '1-2 ребенка'; от 3 и более - 'Многодетные'

In [23]:
# Напишем функцию availability_children:
def availability_children(children):
    if children == 0:
        return 'Бездетные'
    elif 0 < children < 3:
        return '1-2 ребенка'
    return 'Многодетные'
# Проверим результат работы функции availability_children:
a = availability_children(3)
print(a)


Многодетные


Функция работает корректно.

In [24]:
# Создадим отдельный столбец 'children_group', в котором значения - это группировка клиентов банка, 
#исходя из количества детей
# Вызовем метод apply(): 
#берет значения столбца 'children' и применяет к ним функцию availability_children из своего аргумента
df['children_group'] = df['children'].apply(availability_children)
# Выведем статистику по группам:
df['children_group'].value_counts()


Бездетные      14167
1-2 ребенка     6907
Многодетные      380
Name: children_group, dtype: int64

В итоге: данные столбца 'children' - количество детей разбиты на категории: 'Бездетные', '1-2 ребенка', 'Многодетные'. 

Как видим из статистики: большинство заемщиков не имеют детей.     

## Ответы на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Думаю, что клиенты, у которых есть дети чаще не возвращают кредит в срок. Проверим данное предположение.

In [25]:
# Рассмотрим столбец 'debt'. В нем данные логического типа: 0 - не было просрочки по кредиту, 1 - задолженность была
# Убедимся, что в столбце только значение 0 и 1:
print(df['debt'].unique())
# Создадим сводную таблицу result_children с помощью метода pivot_table, аргументами которой будут:
# index = 'children_group', columns = 'debt', values = 'children', aggfunc = count():
result_children = df.pivot_table(index = ['children_group'], columns='debt', values = 'children', aggfunc = 'count', 
                                 margins=True)
#print(result_children)
# Добавим к сводной таблице столбец percent_debt с расчетами: 
# какой процент составляет количество просрочек в каждой группе 
pd.options.display.float_format = '{:.2%}'.format # Зададим формат, чтобы результат в новом столбце был выведен в %
result_children['percent_yes_debt'] = result_children[1] / result_children['All']
#переименуем названия столбцов:
result_children.columns = ['no_debt', 'yes_debt', 'sum_total', 'percent_yes_debt']
result_children # Выведем статистику на экран

[0 1]


,no_debt,yes_debt,sum_total,percent_yes_debt
children_group,,,,
1-2 ребенка,6268,639,6907,9.25%
Бездетные,13096,1071,14167,7.56%
Многодетные,349,31,380,8.16%
All,19713,1741,21454,8.12%


### Вывод

Как видно из результатов статистики: предположение оказалось верным. Больше всего не возвращают кредит в срок клиенты, у которых есть дети.

Большинство заемщиков не имеют детей. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?


Предположим: что люди, у которых семейное положение 'Не женат / не замужем'  чаще не возвращают кредит в срок.

In [26]:
# создадим сводную таблицу total_family с помощью метода pivot_table, аргументами которой будут:
# index = 'family_status', columns = 'debt', values = 'family_status_id', aggfunc = count():
total_family = df.pivot_table(index = ['family_status'], columns = 'debt', values = 'family_status_id', 
                              aggfunc = 'count',
                              margins = True)
pd.options.display.float_format = '{:.2%}'.format # Зададим формат, чтобы результат в новом столбце был выведен в %
total_family['percent'] = total_family[1] / total_family['All']
total_family.columns = ['no_debt', 'yes_debt', 'sum_total', 'percent_yes_debt']
total_family

,no_debt,yes_debt,sum_total,percent_yes_debt
family_status,,,,
Не женат / не замужем,2536,274,2810,9.75%
в разводе,1110,85,1195,7.11%
вдовец / вдова,896,63,959,6.57%
гражданский брак,3763,388,4151,9.35%
женат / замужем,11408,931,12339,7.55%
All,19713,1741,21454,8.12%


### Вывод

Как видим из результатов статистики: предположение наше оказалось верно. Самый большой процент не возврата у клиентов, у которых семейный статус: Не женат / не замужем. Большинство клиентов банка находятся в официальном браке и у них процент не возврата меньше. Видимо это связано с тем, что уровень дохода у таких людей возрастает вдвое.   

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?


Предположим, что люди с очень высоким уровнем дохода реже не возвращают кредит в срок.

In [27]:
total_income_group = df.pivot_table(index = ['income_group'], columns = 'debt', values = 'total_income_end', 
                                    aggfunc = 'count',
                              margins=True)
pd.options.display.float_format = '{:.2%}'.format # Зададим формат, чтобы результат в новом столбце был выведен в %
total_income_group['total_income_group_percent'] = total_income_group[1] / total_income_group['All']
total_income_group.columns = ['no_debt', 'yes_debt', 'sum_total', 'percent_yes_debt']# переименуем названия столбцов
total_income_group

,no_debt,yes_debt,sum_total,percent_yes_debt
income_group,,,,
высокий,4719,438,5157,8.49%
низкий,4898,464,5362,8.65%
очень высокий,5182,388,5570,6.97%
средний,4914,451,5365,8.41%
All,19713,1741,21454,8.12%


### Вывод

Наше предположение верно: у людей с очень высоким уровнем дохода процент просрочек по кредиту меньше. Самый высокий процент не возврата у людей с низким  уровнем дохода. Чтобы сказать с чем это связано, нужно проводить дополнительные исследования: например, изучить какой процент составляют пары, находящихся в официальном браке в данных категориях.

- Как разные цели кредита влияют на его возврат в срок?


Предположим, что самый большой процент не возврата кредита в срок, у которых цель кредита: жилье, недвижимость.

In [28]:
purpose_pivot = df.pivot_table(index = ['conversion_two'], columns = 'debt', values = 'purpose', aggfunc = 'count',
                              margins=True)
pd.options.display.float_format = '{:.2%}'.format # Зададим формат, чтобы результат в новом столбце был выведен в %
purpose_pivot['percent_yes_debt'] = purpose_pivot[1] / purpose_pivot['All']
purpose_pivot.columns = ['no_debt', 'yes_debt', 'sum_total', 'percent_yes_debt']# переименуем названия столбцов
purpose_pivot

,no_debt,yes_debt,sum_total,percent_yes_debt
conversion_two,,,,
автомобиль,3903,403,4306,9.36%
жилье,4152,308,4460,6.91%
недвижимость,5877,474,6351,7.46%
образование,3643,370,4013,9.22%
свадьба,2138,186,2324,8.00%
All,19713,1741,21454,8.12%


### Вывод

Наше предположение оказалось не верным. Самый большой процент не возврата кредита в срок, у людей с целью кредита - автомобиль. На втором месте - образование. Самый низкий кредит не возврата у людей, цель которых - покупка жилья. 

## Общий вывод:

Исходя из всего вышесказанного, делаем вывод: семейное положение и количество детей клиента влияет на факт погашения кредита. Чаще всего берут кредит люди с семейным положением: женат / замужем  и у которых нет детей. У таких пар процент не возврата кредита в срок меньше по сравнению с другими группами. Таким людям нужно чаще предлагать различные кредитные продукты банка.



